<a href="https://colab.research.google.com/github/QaziSaim/CASE-STUDIES/blob/main/Laggraph_InMemoryStore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install langgraph

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig
from typing import Annotated
from typing_extensions import TypedDict
from operator import add

In [ ]:
class State(TypedDict):
  foo:str
  bar:Annotated[list[str], add]

def node_a(state:State):
  return {'foo':'a','bar':['a']}

def node_b(state:State):
  return {'foo':'b','bar':['b']}

workflow = StateGraph(State)
workflow.add_node(node_a)
workflow.add_node(node_b)
workflow.add_edge(START, 'node_a')
workflow.add_edge('node_a','node_b')
workflow.add_edge('node_b',END)

checkpointer = InMemorySaver()
graph = workflow.compile(checkpointer=checkpointer)

config: RunnableConfig = {'configurable':{'thread_id':'1'}}
graph.invoke({'foo':''},config)

{'foo': 'b', 'bar': ['a', 'b']}

In [ ]:
print(graph.get_state(config))

StateSnapshot(values={'foo': 'b', 'bar': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0bb0bb-2777-6177-8002-fdd23b3c5fe1'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-11-06T12:25:36.169171+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0bb0bb-2771-632f-8001-f8e97af0a716'}}, tasks=(), interrupts=())


In [ ]:
list(graph.get_state_history(config))

[StateSnapshot(values={'foo': 'b', 'bar': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0bb0bb-2777-6177-8002-fdd23b3c5fe1'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-11-06T12:25:36.169171+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0bb0bb-2771-632f-8001-f8e97af0a716'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'foo': 'a', 'bar': ['a']}, next=('node_b',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0bb0bb-2771-632f-8001-f8e97af0a716'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-11-06T12:25:36.166755+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0bb0bb-2760-693f-8000-823ae8b58bb1'}}, tasks=(PregelTask(id='341ff800-da6d-2993-da88-abdc4859b685', name='node_b', path=('__pregel_pull', 'node_b'), error=None, interrupts